In [25]:
import cv2
import numpy as np
import pandas as pd
import random
import matplotlib.colors as mcolors
import sys
# import matplotlib.pyplot as plt
# %matplotlib inline

In [26]:
aruco = cv2.aruco
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

In [27]:
# debug情報を出すか
IS_DEBUG = False

cap = cv2.VideoCapture("src/mov/test06.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))    # 動画のfps
current_frame = 1   # 現在の動画フレーム
# マーカーサイズ
marker_length = 0.0255  # [m]
# マーカーの辞書選択
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

# キャリブレーションデータの選択
camera_matrix = np.load("calibration/iphonese/mtx.npy")
distortion_coeff = np.load("calibration/iphonese/dist.npy")

In [28]:
def apllyFilters(img):
    k = 3
    sharp_kernel = np.array([
        [-k / 9, -k / 9, -k / 9],
        [-k / 9, 1 + 8 * k / 9, k / 9],
        [-k / 9, -k / 9, -k / 9]
    ], np.float32)
    img_sp = cv2.filter2D(img, -1, sharp_kernel).astype("uint8")
    return img_sp


In [29]:
capture_ids = [0, 1, 2, 3]
d = 10  # 中心マーカーの直径

In [30]:
while True:
    ret, img = cap.read()
    if not ret or cv2.waitKey(int(1000 / fps)) == 27:
        break
    # img = apllyFilters(img)

    # マーカー認識
    corners, ids, rejectedImgPoints = aruco.detectMarkers(img, dictionary)
    # マーカー可視化
    aruco.drawDetectedMarkers(img, corners, ids, (0, 255, 255))
    
    ids_list = np.squeeze(ids)
    capture_corners_indexes = list(map(lambda i : np.where(ids_list==i)[0][0] if len(np.where(ids_list==i)[0]) == 1 else None, capture_ids))
    for i in capture_corners_indexes:
        if i is not None:
            capture_corners = corners[i][0].astype(int)
            cpx, cpy = np.average(capture_corners, axis=0).astype(int)
            # cv2.circle(img, (cpx,cpy),5, (0,255,255), -1) 

    cv2.imshow('drawDetectedMarkers', img)

    ###debug
    # if current_frame > 50:
    #     break

# 全フレーム終了後の処理

cv2.destroyAllWindows()


In [31]:
ids

array([[10],
       [11],
       [12],
       [14],
       [13],
       [ 7],
       [ 9],
       [ 8],
       [ 6],
       [ 5],
       [ 4],
       [ 3],
       [ 2],
       [ 1],
       [ 0]], dtype=int32)

In [32]:
capture_ids = [14, 1, 2]

list(map(lambda i : np.where(ids_list==i)[0][0] if len(np.where(ids_list==i)[0]) == 1 else None, capture_ids))


[3, 13, 12]